In [1]:
import pandas as pd
import numpy as np
import itertools

In [2]:
λ1 = 2
λ2 = 0.00001
λ3 = 0.00001
μ = 101
#λ1 = 5
#λ2 = 2
#λ3 = 3
#μ = 12
arrivals=1000000

# function to generate independent exponential random variables with mean 1/μ and the size of the arrivals
def generateTimes(x,arrivals):
    return np.random.exponential(scale=1/x,size=arrivals)

def generatePoisson(x,arrivals):
    return np.random.poisson(x,arrivals)

serviceTimes=generateTimes(μ,arrivals)
interarrivalTimeOne=generateTimes(λ1,arrivals)
interarrivalTimeOne[0:1] = 0
interarrivalTimeTwo=generateTimes(λ2,arrivals)
interarrivalTimeTwo[0:1] = 0
interarrivalTimeThree=generateTimes(λ3,arrivals)
interarrivalTimeThree[0:1] = 0

In [3]:
def computeInterarrivals(timeOne,timeTwo,timeThree, arrivals):
    arrivalRange=np.arange(arrivals)
    arrivalArray=np.zeros(int(arrivals))
    interarrivalOne=0
    interarrivalTwo=0
    interarrivalThree=0
    
    for i,λ1,λ2,λ3 in zip(arrivalRange,timeOne,timeTwo,timeThree):
        if λ1<=λ2 and λ1<=λ3:
            interarrivalOne+=1
            arrivalRange[i]=λ1
            arrivalArray[i]=λ1
        elif λ2<=λ1 and λ2<=λ3:
            interarrivalTwo+=1
            arrivalRange[i]=λ2
            arrivalArray[i]=λ2
        elif λ3<=λ1 and λ3<=λ2:
            interarrivalThree+=1
            arrivalRange[i]=λ2
            arrivalArray[i]=λ3

    return interarrivalOne/arrivals,interarrivalTwo/arrivals,interarrivalThree/arrivals,arrivalArray

In [4]:
# return cumulative sum
def interarrivalCumSum(x):
    return np.cumsum(x)

In [5]:
# mesh different incoming 
percentageOne,percentageTwo,percentageThree,interarrivals= computeInterarrivals(interarrivalTimeOne,interarrivalTimeTwo,interarrivalTimeThree, arrivals)
interarrivalsCumulated=interarrivalCumSum(interarrivals)

In [6]:
# calculate times
def calculateTimes(serviceTimes,arrivalTime):
    
    # append extra index to arrival time
    arrivalTime= np.append(arrivalTime,[0])
    
    # append extra index to time Service Begins
    timeServiceBegins=np.zeros(np.size(serviceTimes))
    timeServiceBegins=np.append(timeServiceBegins,[0])
    
    timeServiceEnds=np.zeros(np.size(serviceTimes))
    
    # index list
    indexList=np.arange(np.size(serviceTimes))
    
    for i,b,c,d,e in zip(indexList, arrivalTime, timeServiceBegins, serviceTimes, timeServiceEnds):
        if i!=0:
            if arrivalTime[i]>=timeServiceEnds[i-1]:
                timeServiceBegins[i]=arrivalTime[i]
                timeServiceEnds[i]+=arrivalTime[i]+serviceTimes[i]
            elif arrivalTime[i]<=timeServiceEnds[i-1]:
                timeServiceBegins[i]=timeServiceEnds[i-1]
                timeServiceEnds[i]+=timeServiceEnds[i-1]+serviceTimes[i]
        else:
            timeServiceEnds[0]=serviceTimes[0]

    # waiting time
    waitingTime=timeServiceBegins-arrivalTime
    waitingTime[waitingTime<0]=0
    
    # time of customer in system
    customerInSystem=timeServiceEnds-arrivalTime[:-1]
    
    # idle time of server
    # index shift up and equalizing last index
    idleTime=np.roll(timeServiceBegins[:-1],-1)-timeServiceEnds
    idleTime=np.cumsum(idleTime)
    length=np.size(idleTime)-1
    idleTime[length]=0
        
                
    return timeServiceBegins,timeServiceEnds,waitingTime,customerInSystem, idleTime

In [7]:
timeServiceBegins,timeServiceEnds,waitingTime,customerInSystem,idleTime=calculateTimes(serviceTimes,interarrivalsCumulated)

avgCustomerInSystem=np.mean(customerInSystem)
avgWaitingTime=np.mean(waitingTime)


In [8]:
# data frame for the table
df=pd.DataFrame(columns=['Customer','Interarrival time(minutes)','Arrival time on clock','Service time(minutes)','Time Service begins','Waiting time in Queue','Time service ends','Time customer in system','Idle time of server'])
df.loc[:, 'Customer'] = np.arange(1,(np.size(serviceTimes)+1),1,dtype=int)
df.loc[:, 'Interarrival time(minutes)'] = interarrivals
df.loc[:, 'Arrival time on clock'] = interarrivalsCumulated
df.loc[:, 'Service time(minutes)'] = serviceTimes
df.loc[:, 'Time Service begins'] = timeServiceBegins[:-1]
df.loc[:, 'Time service ends'] = timeServiceEnds
df.loc[:, 'Waiting time in Queue'] = waitingTime[:-1]
df.loc[:, 'Time customer in system'] = customerInSystem
df.loc[:, 'Idle time of server'] = idleTime

df


,Customer,Interarrival time(minutes),Arrival time on clock,Service time(minutes),Time Service begins,Waiting time in Queue,Time service ends,Time customer in system,Idle time of server
0,1,0.000000,0.000000,0.018241,0.000000,0.000000,0.018241,0.018241,0.290679
1,2,0.308920,0.308920,0.000871,0.308920,0.000000,0.309791,0.000871,1.328993
2,3,1.039184,1.348104,0.000403,1.348104,0.000000,1.348507,0.000403,1.582301
3,4,0.253711,1.601815,0.027414,1.601815,0.000000,1.629228,0.027414,1.582301
4,5,0.013487,1.615302,0.002413,1.629228,0.013926,1.631641,0.016339,2.480928
...,...,...,...,...,...,...,...,...,...
999995,999996,0.143954,499686.123194,0.006798,499686.123194,0.000000,499686.129991,0.006798,489798.190028
999996,999997,0.139748,499686.262942,0.007833,499686.262942,0.000000,499686.270775,0.007833,489798.482092
999997,999998,0.299898,499686.562839,0.008542,499686.562839,0.000000,499686.571381,0.008542,489799.163770
999998,999999,0.690219,499687.253058,0.013398,499687.253058,0.000000,499687.266457,0.013398,489799.890501


In [9]:
print("Average nr of customers in system: ",avgCustomerInSystem)
print("Average waiting time: ",avgWaitingTime)
print("Percentage of cars arriving from road 1: ",percentageOne)
print("Percentage of cars arriving from road 2: ",percentageTwo)
print("Percentage of cars arriving from road 3: ",percentageThree)

Average nr of customers in system:  0.010089048437807082
Average waiting time:  0.0002009416910616336
Percentage of cars arriving from road 1:  0.999993
Percentage of cars arriving from road 2:  2e-06
Percentage of cars arriving from road 3:  5e-06
